<a href="https://colab.research.google.com/github/PranavVerma-droid/Arduino-IDE-Repo/blob/main/Projects/AI/LLaMa2/Llama2_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama2 Notebook

This Notebook has been made to train the Llama2 Model by Meta.
This was made to Run in Google Colab, in the T GPU.

The Files for this Model can Be Found on [HuggingFace](https://huggingface.co/PranavVerma-droid/llama-2-7b-trained).

### Installing Dependencies

In [ ]:
# Install all Dependencies

!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
# Import all Nessesary Libraries to the Runtime in Google Colab.

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    set_seed,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

### Specify all Parameters

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/llama-2-7b-chat-hf"
# model_name = "PranavVerma-droid/llama-2-7b-trained"

# The instruction dataset to use from HuggingFace

# dataset_name = "mlabonne/guanaco-llama2-1k"
dataset_name = "PranavVerma-droid/llama2-7b-training"

# Model Name to Use
new_model = "llama-2-7b-trained"

# Output directory where the model predictions and checkpoints will be stored in Google Colab
output_dir = "./results"


num_train_epochs = 1
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

fp16 = False
bf16 = False

per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
save_steps = 25
logging_steps = 25
max_seq_length = None

packing = False
device_map = {"": 0}

### Training the Model

In [ ]:
# PLEASE CHANGE THE MACHINE TYPE IN GOOGLE COLAB BEFORE TRAINING YOUR AI, SET IT TO T4 GPU,
# OR ELSE THIS WILL NOT WORK.

# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

### Tensorboard Plots

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

### Using the Model

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

prompt = input("You: ")

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")

response = result[0]['generated_text'].split("\n")[-1].strip()
print(f"AI: {response}")

### Storing the Model Somewhere for HuggingFace

In [ ]:
# We need to merge the weights from LoRA with the base model. Unfortunately, as far as I know, there is no straightforward way to do it:
#  we need to reload the base model in FP16 precision and use the peft library to merge everything.
# Alas, it also creates a problem with the VRAM (despite emptying it), so I recommend RESTARTING THE RUNTIME ( Runtime < Restart Runtime ),
# RE-EXECUTING THE THREE FIRST CELLS, and then EXECUTING THIS ONE.

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#### Pushing the Model to Hugging Face

In [ ]:
# After you run the Above Storing code, Run this Cell to Push Everything to HuggingFace.
# Do not use the Token as a Git Credential.

!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)